In [4]:
import numpy as np
from PIL import Image

In [5]:
def image_open():
    image = Image.open('image.tif')
    image_array = np.array(im)
    return image_array

In [6]:
def main():
    print("Program starts.")
    return 0

In [7]:
main()